# Meilisearch

> [Meilisearch](https://meilisearch.com) 是一个开源、闪电般快速且高度相关的搜索引擎。它提供了许多出色的默认设置，可帮助开发人员构建响应迅速的搜索体验。
>
> 您可以[自行托管 Meilisearch](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation) 或在 [Meilisearch Cloud](https://www.meilisearch.com/pricing) 上运行。

Meilisearch v1.3 支持向量搜索。本页面将指导您如何集成 Meilisearch 作为向量存储，并利用它执行向量搜索。

您需要安装 `langchain-community`（使用 `pip install -qU langchain-community`）才能使用此集成。

## 设置

### 启动 Meilisearch 实例

您需要一个运行中的 Meilisearch 实例作为您的向量存储。您可以选择在本地运行 [Meilisearch](https://www.meilisearch.com/docs/learn/getting_started/installation#local-installation)，或者创建一个 [Meilisearch Cloud](https://cloud.meilisearch.com/) 账户。

从 Meilisearch v1.3 开始，向量存储是一项实验性功能。启动 Meilisearch 实例后，您需要**启用向量存储**。对于自托管的 Meilisearch，请阅读关于[启用实验性功能](https://www.meilisearch.com/docs/learn/experimental/overview)的文档。在 **Meilisearch Cloud** 上，请通过您的项目“设置”页面启用 _Vector Store_。

现在您应该拥有一个已启用向量存储的运行中的 Meilisearch 实例了。🎉

### 凭证

为了与您的 Meilisearch 实例进行交互，Meilisearch SDK 需要一个主机（实例的 URL）和一个 API 密钥。

**主机**

- 在**本地**，默认主机是 `localhost:7700`
- 在 **Meilisearch Cloud** 上，请在您的项目“设置”页面查找主机

**API 密钥**

Meilisearch 实例开箱即用地为您提供了三个 API 密钥：
- `MASTER KEY` — 仅用于创建您的 Meilisearch 实例
- `ADMIN KEY` — 仅在服务器端使用，用于更新您的数据库及其设置
- `SEARCH KEY` — 一个可以安全地在前端应用程序中共享的密钥

您可以根据需要创建[额外的 API 密钥](https://www.meilisearch.com/docs/learn/security/master_api_keys)。

### 安装依赖

本指南使用 [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python)。你可以通过运行以下命令进行安装：

In [ ]:
%pip install --upgrade --quiet  meilisearch

有关更多信息，请参阅 [Meilisearch Python SDK 文档](https://meilisearch.github.io/meilisearch-python/)。

## 示例

初始化 Meilisearch 向量存储有多种方法：提供 Meilisearch 客户端或根据需要提供 URL 和 API 密钥。在我们的示例中，凭据将从环境变量加载。

您可以使用 `os` 和 `getpass` 在 Notebook 环境中提供环境变量。您可以在以下所有示例中使用此技术。

In [ ]:
import getpass
import os

if "MEILI_HTTP_ADDR" not in os.environ:
    os.environ["MEILI_HTTP_ADDR"] = getpass.getpass(
        "Meilisearch HTTP address and port:"
    )
if "MEILI_MASTER_KEY" not in os.environ:
    os.environ["MEILI_MASTER_KEY"] = getpass.getpass("Meilisearch API Key:")

我们要使用 OpenAIEmbeddings，因此必须获取 OpenAI API 密钥。

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### 添加文本和嵌入

此示例将文本添加到 Meilisearch 向量数据库，而无需初始化 Meilisearch 向量存储。

In [ ]:
from langchain_community.vectorstores import Meilisearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

embeddings = OpenAIEmbeddings()
embedders = {
    "default": {
        "source": "userProvided",
        "dimensions": 1536,
    }
}
embedder_name = "default"

In [ ]:
with open("../../how_to/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
# Use Meilisearch vector store to store texts & associated embeddings as vector
vector_store = Meilisearch.from_texts(
    texts=texts, embedding=embeddings, embedders=embedders, embedder_name=embedder_name
)

在后台，Meilisearch 会将文本转换为多个向量。这将为我们带来与以下示例相同的结果。

### 添加文档和嵌入

在此示例中，我们将使用 Langchain TextSplitter 来将文本拆分为多个文档。然后，我们将这些文档及其嵌入一同存储。

In [ ]:
from langchain_community.document_loaders import TextLoader

# Load text
loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Create documents
docs = text_splitter.split_documents(documents)

# Import documents & embeddings in the vector store
vector_store = Meilisearch.from_documents(
    documents=documents,
    embedding=embeddings,
    embedders=embedders,
    embedder_name=embedder_name,
)

# Search in our vector store
query = "What did the president say about Ketanji Brown Jackson"
docs = vector_store.similarity_search(query, embedder_name=embedder_name)
print(docs[0].page_content)

## 通过创建 Meilisearch Vectorstore 添加文档

在此方法中，我们创建一个向量存储对象并将文档添加到其中。

In [ ]:
import meilisearch
from langchain_community.vectorstores import Meilisearch

client = meilisearch.Client(url="http://127.0.0.1:7700", api_key="***")
vector_store = Meilisearch(
    embedding=embeddings,
    embedders=embedders,
    client=client,
    index_name="langchain_demo",
    text_key="text",
)
vector_store.add_documents(documents)

## 相似度搜索及评分

此特定方法允许您返回文档以及查询与它们之间的距离评分。`embedder_name` 是应用于语义搜索的 embedder 的名称，默认为“default”。

In [ ]:
docs_and_scores = vector_store.similarity_search_with_score(
    query, embedder_name=embedder_name
)
docs_and_scores[0]

## 向量相似性搜索
`embedder_name` 是应用于语义搜索的 embedding 的名称，默认为 "default"。

In [ ]:
embedding_vector = embeddings.embed_query(query)
docs_and_scores = vector_store.similarity_search_by_vector(
    embedding_vector, embedder_name=embedder_name
)
docs_and_scores[0]

## 附加资源

文档
- [Meilisearch](https://www.meilisearch.com/docs/)
- [Meilisearch Python SDK](https://python-sdk.meilisearch.com)

开源代码库
- [Meilisearch 仓库](https://github.com/meilisearch/meilisearch)
- [Meilisearch Python SDK](https://github.com/meilisearch/meilisearch-python)